# Test the pathline code

In [ ]:
import numpy as np
import pylab as plt
import xarray as xr
from pypism.pathlines import compute_trajectory, pathlines_to_geopandas
from pypism.interpolation import velocity_at_point
from shapely import Point
from joblib import Parallel, delayed
import geopandas as gp
from typing import Union
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
def create_circular() -> xr.Dataset:
    """
    Create xr.Dataset with radial velocity field
    """
    time = pd.date_range("2000-01-01", periods=1)
    reference_time = pd.Timestamp("2000-01-01")

    nx = 20_001
    ny = 20_001
    x = np.linspace(-100e3, 100e3, nx)
    y = np.linspace(-100e3, 100e3, ny)
    X, Y = np.meshgrid(x, y)

    # Directional vectors
    vx = -Y / np.sqrt(X**2 + Y**2) * 250
    vy = X / np.sqrt(X**2 + Y**2) * 250
    v = np.sqrt(vx**2 + vy**2)

    vx = vx.reshape(1, ny, nx)
    vy = vy.reshape(1, ny, nx)
    v = v.reshape(1, ny, nx)

    v_err = v / 10
    vx_err = np.abs(vx / 20)
    vy_err = np.abs(vy / 20)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
        "time": (["time"], time, {}),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v": xr.DataArray(
                data=v,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
            "vx_err": xr.DataArray(
                data=vx_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy_err": xr.DataArray(
                data=vy_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v_err": xr.DataArray(
                data=v_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    
    return ds


In [ ]:
ds = create_circular()
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())
V = np.squeeze(ds["v"].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)

## Create doc-string examples

In [ ]:
import numpy as np
from pypism.geom import Point

nx = 201
ny = 401
x = np.linspace(-100e3, 100e3, nx)
y = np.linspace(-100e3, 100e3, ny)
X, Y = np.meshgrid(x, y)

# Directional vectors
vx = -Y / np.sqrt(X**2 + Y**2) * 250
vy = X / np.sqrt(X**2 + Y**2) * 250

p = Point(0, -50000)

# pts, pts_error_estim = compute_trajectory(p, vx, vx, x, y, dt=1, total_time=10)

In [ ]:
%%time

compute_trajectory(p, vx, vx, x, y, dt=0.1, total_time=1_000)

In [ ]:
starting_point = Point(0, -1_000)
r = starting_point.distance(Point(0, 0))
circ = 2 * r * np.pi
vx, vy = velocity_at_point(Vx, Vy, x, y, starting_point)
v = np.sqrt(vx**2 + vy**2)
total_time = circ / v    

dts = np.logspace(-3, 2, 6)
dt_trajs = {}

progress = tqdm(dts, total=len(dts), leave=False, position=0)
for dt in progress:
    progress.set_description(f"Time step {dt}")
    pts, pts_error_estim = compute_trajectory(starting_point, Vx, Vy, x, y, total_time=total_time+dt, dt=dt, reverse=True)
    dt_trajs[dt] = pts


In [ ]:
a.distance(b)

In [ ]:
def create_linear() -> xr.Dataset:
    """
    Create xr.Dataset with radial velocity field
    """
    time = pd.date_range("2000-01-01", periods=1)
    reference_time = pd.Timestamp("2000-01-01")

    nx = 201
    ny = 201
    x_min = -1
    x_max = 1
    y_min = -1
    y_max = 1
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    # Directional vectors
    vx = X
    vy = -Y
    v = np.sqrt(vx**2 + vy**2)

    vx = vx.reshape(1, ny, nx)
    vy = vy.reshape(1, ny, nx)
    v = v.reshape(1, ny, nx)

    v_err = v / 10
    vx_err = np.abs(vx / 20)
    vy_err = np.abs(vy / 20)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
        "time": (["time"], time, {}),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v": xr.DataArray(
                data=v,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
            "vx_err": xr.DataArray(
                data=vx_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy_err": xr.DataArray(
                data=vy_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v_err": xr.DataArray(
                data=v_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    
    return ds


In [ ]:
ds = create_linear()

$$\frac{\mathrm{d}\mathbf{r}}{\mathrm{d}t} = \mathbf{u} = x\mathbf{i}  - y \mathbf{j} $$

$$\mathbf{r}(t) = \mathbf{r}_0 + \int_{0}^{t'} \mathbf{u}(t') \mathrm{d}t'$$

$$\mathbf{r}(t) = \mathbf{r}(t_0) \left(e^{t}\mathbf{i} + e^{-t}\mathbf{j}\right)$$

In [ ]:
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())
V = np.squeeze(ds["v"].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)
total_time = np.exp(1)
starting_point = Point(0.05, 0.95)
pts, pts_error_estim = compute_trajectory(starting_point, Vx, Vy, x, y, total_time=total_time, dt=dt)

In [ ]:
def exact_solution(x0, t):
    x = x0.x * np.exp(t)
    y = x0.y * np.exp(-t)
    return Point(x, y)

In [ ]:
r_exact = exact_solution(starting_point, total_time)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ds["v"].plot(ax=ax)
ax.scatter([p.x for p in pts], [p.y for p in pts], c="k", s=2)
ax.scatter(r_exact.x, r_exact.y, c="r")

In [ ]:
from numpy import ndarray

In [ ]:
def interpolate(ds, points, x_var="vx", y_var="vy"):
    return ds[x_var].interp(points), ds[y_var].interp(points)

In [ ]:
from typing import Tuple
from pypism.interpolation import InterpolationMatrix

In [ ]:
n = 1_0000
sample_points_array = np.random.rand(n, 2)
sample_points = [Point(x, y) for x,y in sample_points_array]
sample_points_xr = {"x": sample_points_array[:, 0], "y": sample_points_array[:, 1]}

In [ ]:
%timeit interpolate(ds, sample_points_xr)

In [ ]:
%timeit velocity_at_point(Vx, Vy, x, y, sample_points)

In [ ]:
from shapely import Point

In [ ]:
def interpolate_at_point(
    Vx: ndarray,
    Vy: ndarray,
    x: ndarray,
    y: ndarray,
    px: float,
    py: float,
) -> Tuple:
    """
    Return velocity at Point p using bilinear interpolation
    """

    A = InterpolationMatrix(x, y, px, py)
    vx = A.apply(Vx)
    vy = A.apply(Vy)
    return vx, vy


In [ ]:
%timeit interpolate_at_point(Vx, Vy, x, y, sample_points_array[:, 0], sample_points_array[:, 1])

In [ ]:
VX = ds["vx"].where((x<0) & (y<0), np.nan)
VY = ds["vy"].where((x<0) & (y<0), np.nan)

Vx = np.squeeze(VX.to_numpy())
Vy = np.squeeze(VY.to_numpy())
x = VX["x"].to_numpy()
y = VY["y"].to_numpy()


In [ ]:
interpolate_at_point(Vx, Vy, x, y, 0, 0)

In [ ]:
interpolate_at_point(Vx, Vy, x, y, -1, -1)

In [ ]:
np.array([1, 2, 3]).reshape(1, -1)

In [ ]:
pt = interpolate_at_point(Vx, Vy, x, y, -1, -1)

In [ ]:
delta_time =1 

In [ ]:
    def k2_v(p, k1_v):
        return p + (0.25) * delta_time * k1_v


In [ ]:
k2_v(pt, pt)

In [ ]:
isinstance([1], numbers.Number)

In [ ]:
        def to_array(data):        
            if isinstance(data, numbers.Number):
                data = np.array([data])
            elif isinstance(data, list):
                data = np.array(data)
            return data


In [ ]:
Vx

In [ ]:
class InterpolationMatrix:
    """Stores bilinear and nearest neighbor interpolation weights used to
    extract profiles.

    """

    def __init__(
        self,
        x: ndarray,
        y: ndarray,
        px: Union[float, list, ndarray],
        py: Union[float, list, ndarray],
        bilinear: bool = True,
    ):
        """Interpolate values of z to points (px,py) assuming that z is on a
        regular grid defined by x and y."""
        super().__init__()

        def to_array(data):        
            if isinstance(data, numbers.Number):
                data = np.array([data])
            elif isinstance(data, list):
                data = np.array(data)
            return data

        px = to_array(px)
        py = to_array(py)
            
        assert px.size == py.size

        # The grid has to be equally spaced.
        assert np.fabs(np.diff(x).max() - np.diff(x).min()) < 1e-9
        assert np.fabs(np.diff(y).max() - np.diff(y).min()) < 1e-9

        dx = x[1] - x[0]
        dy = y[1] - y[0]

        assert dx != 0
        assert dy != 0

        cs = [self.grid_column(x, dx, p_x) for p_x in px]
        rs = [self.grid_column(y, dy, p_y) for p_y in py]

        self.c_min = np.min(cs)
        self.c_max = min(np.max(cs) + 1, len(x) - 1)

        self.r_min = np.min(rs)
        self.r_max = min(np.max(rs) + 1, len(y) - 1)

        # compute the size of the subset needed for interpolation
        self.n_rows = self.r_max - self.r_min + 1
        self.n_cols = self.c_max - self.c_min + 1

        n_points = len(px)
        self.A = scipy.sparse.lil_matrix((n_points, self.n_rows * self.n_cols))

        if bilinear:
            self._compute_bilinear_matrix(x, y, dx, dy, px, py)
        else:
            raise NotImplementedError

    def column(self, r, c):
        """Interpolation matrix column number corresponding to r,c of the
        array *subset*. This is the same as the linear index within
        the subset needed for interpolation.

        """
        return self.n_cols * min(r, self.n_rows - 1) + min(c, self.n_cols - 1)

    @staticmethod
    def find(grid, delta, point):
        """Find the point to the left of point on the grid with spacing
        delta."""
        if delta > 0:
            # grid points are stored in the increasing order
            if point <= grid[0]:  # pylint: disable=R1705
                return 0
            elif point >= grid[-1]:
                return len(grid) - 1  # pylint: disable=R1705
            else:
                return int(np.floor((point - grid[0]) / delta))
        else:
            # grid points are stored in the decreasing order
            if point >= grid[0]:  # pylint: disable=R1705
                return 0
            elif point <= grid[-1]:
                return len(grid) - 1
            else:
                return int(np.floor((point - grid[0]) / delta))

    def grid_column(self, x, dx, X):
        "Input grid column number corresponding to X."
        return self.find(x, dx, X)

    def grid_row(self, y, dy, Y):
        "Input grid row number corresponding to Y."
        return self.find(y, dy, Y)

    def _compute_bilinear_matrix(self, x, y, dx, dy, px, py):
        """Initialize a bilinear interpolation matrix."""
        for k in range(self.A.shape[0]):
            x_k = px[k]
            y_k = py[k]
            
            x_min = np.min(x)
            x_max = np.max(x)

            y_min = np.min(y)
            y_max = np.max(y)

            # make sure we are in the bounding box defined by the grid
            x_k = max(x_k, x_min)
            x_k = min(x_k, x_max)
            y_k = max(y_k, y_min)
            y_k = min(y_k, y_max)

            C = self.grid_column(x, dx, x_k)
            R = self.grid_row(y, dy, y_k)

            alpha = (x_k - x[C]) / dx
            beta = (y_k - y[R]) / dy

            if alpha < 0.0:
                alpha = 0.0
            elif alpha > 1.0:
                alpha = 1.0

            if beta < 0.0:
                beta = 0.0
            elif beta > 1.0:
                beta = 1.0

            # indexes within the subset needed for interpolation
            c = C - self.c_min
            r = R - self.r_min

            self.A[k, self.column(r, c)] += (1.0 - alpha) * (1.0 - beta)
            self.A[k, self.column(r + 1, c)] += (1.0 - alpha) * beta
            self.A[k, self.column(r, c + 1)] += alpha * (1.0 - beta)
            self.A[k, self.column(r + 1, c + 1)] += alpha * beta

    def adjusted_matrix(self, mask):
        """Return adjusted interpolation matrix that ignores missing (masked)
        values."""

        A = self.A.tocsr()
        n_points = A.shape[0]

        output_mask = np.zeros(n_points, dtype=np.bool_)

        for r in range(n_points):
            # for each row, i.e. each point along the profile
            row = np.s_[A.indptr[r] : A.indptr[r + 1]]
            # get the locations and values
            indexes = A.indices[row]
            values = A.data[row]

            # if a particular location is masked, set the
            # interpolation weight to zero
            for k, index in enumerate(indexes):
                if np.ravel(mask)[index]:
                    values[k] = 0.0

            # normalize so that we still have an interpolation matrix
            if values.sum() > 0:
                values = values / values.sum()
            else:
                output_mask[r] = True

            A.data[row] = values

        A.eliminate_zeros()

        return A, output_mask

    def apply(self, array):
        """Apply the interpolation to an array. Returns values at points along
        the profile."""
        subset = array[self.r_min : self.r_max + 1, self.c_min : self.c_max + 1]
        pi = self.apply_to_subset(subset)
        print(pi)
        if pi.size == 1:
            return pi.item()
        else:
            return pi

    def apply_to_subset(self, subset):
        """Apply interpolation to an array subset."""

        if np.ma.is_masked(subset):
            A, mask = self.adjusted_matrix(subset.mask)
            data = A * np.ravel(subset)
            return np.ma.array(data, mask=mask)

        return self.A.tocsr() * np.ravel(subset)
